In [1]:
import pandas as pd
import numpy as np 
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.layers import  *

In [4]:
data_df = pd.read_csv('../input/coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv')

In [5]:
data_df.head()

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN


In [6]:
data_df.shape

(5910, 6)

In [7]:
test_dir ='../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test'
train_dir = '../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train'

train_data = data_df[data_df['Dataset_type']=='TRAIN']
test_data = data_df[data_df['Dataset_type']=='TEST']

In [8]:
train_data.head()

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN


In [9]:
test_data.head()

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
5286,5309,IM-0021-0001.jpeg,Normal,TEST,NaN,NaN
5287,5310,IM-0019-0001.jpeg,Normal,TEST,NaN,NaN
5288,5311,IM-0017-0001.jpeg,Normal,TEST,NaN,NaN
5289,5312,IM-0016-0001.jpeg,Normal,TEST,NaN,NaN
5290,5313,IM-0015-0001.jpeg,Normal,TEST,NaN,NaN


In [10]:
train_data.shape

(5286, 6)

In [11]:
test_data.shape

(624, 6)

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
train_gen = train_datagen.flow_from_dataframe(dataframe=train_data,
                                            directory=train_dir,
                                              x_col="X_ray_image_name",
                                            y_col="Label",
                                             target_size=(150, 150),
                                             batch_size=32,
                                             class_mode='binary',
                                              shuffle=True
                                             )
test_gen = test_datagen.flow_from_dataframe(dataframe=test_data,
                                            directory=test_dir,
                                            x_col="X_ray_image_name",
                                            y_col="Label",
                                             target_size=(150, 150),
                                             batch_size=32,
                                             class_mode='binary',
                                              shuffle=True
                                             )

Found 5286 validated image filenames belonging to 2 classes.
Found 624 validated image filenames belonging to 2 classes.


In [14]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'
                      ])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [19]:
history = model.fit_generator(
    train_gen,
    steps_per_epoch=len(train_gen)/32,
    epochs=15,
    validation_data=test_gen,
    validation_steps=30)

Epoch 1/15
6/5 [==================================] - 8s 1s/step - loss: 0.3274 - acc: 0.8802 - val_loss: 0.4497 - val_acc: 0.7628
Epoch 2/15
6/5 [==================================] - 3s 453ms/step - loss: 0.3048 - acc: 0.8854
Epoch 3/15
6/5 [==================================] - 2s 414ms/step - loss: 0.2433 - acc: 0.9427
Epoch 4/15
6/5 [==================================] - 3s 497ms/step - loss: 0.4171 - acc: 0.8073
Epoch 5/15
6/5 [==================================] - 3s 529ms/step - loss: 0.2510 - acc: 0.9010
Epoch 6/15
6/5 [==================================] - 3s 473ms/step - loss: 0.2886 - acc: 0.8958
Epoch 7/15
6/5 [==================================] - 3s 510ms/step - loss: 0.3425 - acc: 0.8490
Epoch 8/15
6/5 [==================================] - 3s 458ms/step - loss: 0.3657 - acc: 0.8438
Epoch 9/15
6/5 [==================================] - 2s 398ms/step - loss: 0.2589 - acc: 0.8854
Epoch 10/15
6/5 [==================================] - 2s 358ms/step - loss: 0.3201 - acc: 0.

In [21]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [22]:
acc , loss , val_acc , val_loss

([0.8802083134651184,
  0.8854166865348816,
  0.9427083134651184,
  0.8072916865348816,
  0.9010416865348816,
  0.8958333134651184,
  0.8489583134651184,
  0.84375,
  0.8854166865348816,
  0.8855421543121338,
  0.8489583134651184,
  0.8177083134651184,
  0.8541666865348816,
  0.9010416865348816,
  0.890625],
 [0.32743459939956665,
  0.3048063814640045,
  0.24328143894672394,
  0.41713079810142517,
  0.25096604228019714,
  0.2886241376399994,
  0.3425346612930298,
  0.3656720221042633,
  0.25889086723327637,
  0.32006555795669556,
  0.34047964215278625,
  0.40878936648368835,
  0.35483646392822266,
  0.25702357292175293,
  0.30473485589027405],
 [0.7628205418586731],
 [0.44965341687202454])

In [ ]:
model.save_weights('./model_weights1.h5')